# RAG Evaluation
_Authored by: [Aymeric Roucher](https://huggingface.co/m-ric)_

This notebook demonstrates how you can evaluate your RAG (Retrieval Augmented Generation), by building a synthetic evaluation dataset and using LLM-as-a-judge to compute the accuracy of your system.

For an introduction to RAG, you can check [this other cookbook](rag_zephyr_langchain)!

RAG systems are complex: here a RAG diagram, where we noted in blue all possibilities for system enhancement:

<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png" height="700">

Implementing any of these improvements can bring a huge performance boost; but changing anything is useless if you cannot monitor the impact of your changes on the system's performance!
So let's see how to evaluate our RAG system.

### Evaluating RAG performance

Since there are so many moving parts to tune with a big impact on performance, benchmarking the RAG system is crucial.

For our evaluation pipeline, we will need:
1. An evaluation dataset with question - answer couples (QA couples)
2. An evaluator to compute the accuracy of our system on the above evaluation dataset.

➡️ It turns out, we can use LLMs to help us all along the way!
1. The evaluation dataset will be synthetically generated by an LLM 🤖, and questions will be filtered out by other LLMs 🤖
2. An [LLM-as-a-judge](https://huggingface.co/papers/2306.05685) agent 🤖 will then perform the evaluation on this synthetic dataset.

__Let's dig into it and start building our evaluation pipeline!__ First, we install the required model dependancies.

In [1]:
!pip install -q torch transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets langchain-community ragatouille

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.

In [2]:
!pip install ragatouille

In [3]:
!pip install jedi==0.16

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.7 MB/s eta 0:00:00


In [ ]:
!pip install requests==2.32.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.5
    Uninstalling requests-2.32.5:
      Successfully uninstalled requests-2.32.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.4.1 requires requests<3.0.0,>=2.32.5, but you have requests 2.32.4 which is incompatible.


In [4]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None)

### Load your knowledge base

In [22]:
ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")

# 1. Build a synthetic dataset for evaluation
We first build a synthetic dataset of questions and associated contexts. The method is to get elements from our knowledge base, and ask an LLM to generate questions based on these documents.

Then we setup other LLM agents to act as quality filters for the generated QA couples: each of them will act as the filter for a specific flaw.

### 1.1. Prepare source documents

In [6]:
!pip install -U langchain-text-splitters langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 1.2.6
    Uninstalling langchain-core-1.2.6:
      Successfully uninstalled langchain-core-1.2.6


In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document as LangchainDocument

langchain_docs = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

  0%|          | 0/2647 [00:00<?, ?it/s]

### 1.2. Setup agents for question generation

We use [Mixtral](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1) for QA couple generation because it it has excellent performance in leaderboards such as [Chatbot Arena](https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard).

In [ ]:
from openai import OpenAI

# Initialize the OpenAI client
# It will automatically look for an environment variable named OPENAI_API_KEY
# Or you can pass it explicitly: client = OpenAI(api_key="your-key-here")
llm_client = OpenAI(api_key="your-api-key")

def call_llm(openai_client: OpenAI, prompt: str):
    # Wrap your prompt in the expected chat dictionary format
    messages = [{"role": "user", "content": prompt}]

    # Call the Chat Completion API
    response = openai_client.chat.completions.create(
        model="gpt-4o",  # or "gpt-3.5-turbo"
        messages=messages,
        max_tokens=1000,
    )

    # Extract the text content from the response object
    return response.choices[0].message.content

# Test the call
result = call_llm(llm_client, "This is a test context")
print(result)

Got it! How can I assist you today?


In [25]:
QA_generation_prompt = """
Your task is to write a high-quality technical question and answer pair based on the given context.

The question must be relevant for a Machine Learning Engineer or Data Scientist working with the Hugging Face ecosystem (Transformers, Datasets, Hub, etc.).

Guidelines:
1. **Focus on Technical Utility**: Instead of simple facts, ask about implementation details, library usage, model characteristics, or troubleshooting mentioned in the context.
2. **Be Standalone**: The question must be understandable without seeing the context. Use specific names (e.g., "the ViT model", "the GLUE dataset") instead of "the model" or "this dataset".
3. **Search Engine Style**: Formulate it like a query someone would type into Google or Stack Overflow.
4. **No Meta-references**: Do NOT use phrases like "according to the text", "in the context", or "the provided passage".
5. **Concise Answer**: The answer should be direct and technically accurate.

Provide your answer as follows:

Output:::
Factoid question: (your technical question)
Answer: (your concise technical answer)

Now here is the context.

Context: {context}\n
Output:::"""

Now let's generate our QA couples.

*   Mục danh sách
*   Mục danh sách


For this example, we generate only 10 QA couples and will load the rest from the Hub.

But for your specific knowledge base, given that you want to get at least ~100 test samples, and accounting for the fact that we will filter out around half of these with our critique agents later on, you should generate much more, in the >200 samples.

In [35]:
import re
import random
from tqdm import tqdm

N_GENERATIONS = 10
print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    output_QA_couple = call_llm(
        llm_client, QA_generation_prompt.format(context=sampled_context.page_content)
    )

    try:
        # 1. Dùng Regex để bắt Question và Answer không phân biệt hoa thường, khoảng trắng
        # Tìm nội dung giữa "Factoid question:" và "Answer:"
        q_pattern = r"Factoid [Qq]uestion:\s*(.*?)(?=\s*Answer:|$)"
        a_pattern = r"Answer:\s*(.*)"

        question_match = re.search(q_pattern, output_QA_couple, re.DOTALL | re.IGNORECASE)
        answer_match = re.search(a_pattern, output_QA_couple, re.DOTALL | re.IGNORECASE)

        if not question_match or not answer_match:
            print(f"Skipped: Không tìm thấy đúng cấu trúc trong output: {output_QA_couple[:50]}...")
            continue

        question = question_match.group(1).strip()
        answer = answer_match.group(1).strip()

        # 2. Nới lỏng giới hạn độ dài (câu trả lời kỹ thuật cần không gian để giải thích)
        if len(answer) > 800:
            print("Skipped: Answer quá dài (> 800 chars)")
            continue

        outputs.append({
            "context": sampled_context.page_content,
            "question": question,
            "answer": answer,
            "source_doc": sampled_context.metadata["source"],
        })

    except Exception as e:
        print(f"Lỗi hệ thống khi xử lý context: {e}")
        continue

print(f"Hoàn thành! Đã tạo được {len(outputs)} cặp QA chất lượng.")

Generating 10 QA couples...


100%|██████████| 10/10 [00:26<00:00,  2.68s/it]

Hoàn thành! Đã tạo được 10 cặp QA chất lượng.


In [36]:
outputs

[{'context': '`max_length`: the maximum sequence length (default value is `128`).\n\n`device`: either "gpu" or "cpu" (default value is `None`). \n\n```python\n>>> results = frugalscore.compute(predictions=[\'hello there\', \'huggingface\'], references=[\'hello world\', \'hugging face\'], batch_size=16, max_length=64, device="gpu")\n```\n\n## Output values\n\nThe output of FrugalScore is a dictionary with the list of scores for each prediction-reference pair:\n```python\n{\'scores\': [0.6307541, 0.6449357]}\n```\n\n### Values from popular papers\nThe [original FrugalScore paper](https://arxiv.org/abs/2110.08559) reported that FrugalScore-Tiny retains 97.7/94.7% of the original performance compared to [BertScore](https://huggingface.co/metrics/bertscore) while running 54 times faster and having 84 times less parameters.\n\n## Examples \n\nMaximal values (exact match between `references` and `predictions`): \n\n```python\n>>> frugalscore = evaluate.load("frugalscore")\n>>> results = fruga

In [37]:
display(pd.DataFrame(outputs).head(1))

,context,question,answer,source_doc
0,"`max_length`: the maximum sequence length (default value is `128`).\n\n`device`: either ""gpu"" or ""cpu"" (default value is `None`). \n\n```python\n>>> results = frugalscore.compute(predictions=['hello there', 'huggingface'], references=['hello world', 'hugging face'], batch_size=16, max_length=64, device=""gpu"")\n```\n\n## Output values\n\nThe output of FrugalScore is a dictionary with the list of scores for each prediction-reference pair:\n```python\n{'scores': [0.6307541, 0.6449357]}\n```\n\n### Values from popular papers\nThe [original FrugalScore paper](https://arxiv.org/abs/2110.08559) reported that FrugalScore-Tiny retains 97.7/94.7% of the original performance compared to [BertScore](https://huggingface.co/metrics/bertscore) while running 54 times faster and having 84 times less parameters.\n\n## Examples \n\nMaximal values (exact match between `references` and `predictions`): \n\n```python\n>>> frugalscore = evaluate.load(""frugalscore"")\n>>> results = frugalscore.compute(predictions=['hello world'], references=['hello world'])\n>>> print(results)\n{'scores': [0.9891098]}\n```\n\nPartial values: \n\n```python\n>>> frugalscore = evaluate.load(""frugalscore"")\n>>> results = frugalscore.compute(predictions=['hello world'], references=['hugging face'])\n>>> print(results)\n{'scores': [0.42482382]}\n```\n\n## Limitations and bias\n\nFrugalScore is based on [BertScore](https://huggingface.co/metrics/bertscore) and [MoverScore](https://arxiv.org/abs/1909.02622), and the models used are based on the original models used for these scores.\n\nThe full list of available models for FrugalScore is:",How can you specify the maximum sequence length and device type when computing FrugalScore in Hugging Face?,"When computing FrugalScore using the `frugalscore.compute()` method, you can specify the maximum sequence length by setting the `max_length` parameter, and you can choose the device type by setting the `device` parameter to either ""gpu"" or ""cpu"". For example: `frugalscore.compute(predictions=['hello there'], references=['hello world'], max_length=64, device=""gpu"")`.",huggingface/evaluate/blob/main/metrics/frugalscore/README.md


### 1.3. Setup critique agents

The questions generated by the previous agent can have many flaws: we should do a quality check before validating these questions.

We thus build critique agents that will rate each question on several criteria, given in [this paper](https://huggingface.co/papers/2312.10003):
- **Groundedness:** can the question be answered from the given context?
- **Relevance:** is the question relevant to users? For instance, `"What is the date when transformers 4.29.1 was released?"` is not relevant for ML practitioners.

One last failure case we've noticed is when a function is tailored for the particular setting where the question was generated, but undecipherable by itself, like `"What is the name of the function used in this guide?"`.
We also build a critique agent for this criteria:
- **Stand-alone**: is the question understandable free of any context, for someone with domain knowledge/Internet access? The opposite of this would be `What is the function used in this article?` for a question generated from a specific blog article.

We systematically score functions with all these agents, and whenever the score is too low for any one of the agents, we eliminate the question from our eval dataset.

💡 ___When asking the agents to output a score, we first ask them to produce its rationale. This will help us verify scores, but most importantly, asking it to first output rationale gives the model more tokens to think and elaborate an answer before summarizing it into a single score token.___

We now build and run these critique agents.

In [38]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [39]:
print("Generating critique for each QA couple...")
for output in tqdm(outputs):
    evaluations = {
        "groundedness": call_llm(
            llm_client,
            question_groundedness_critique_prompt.format(
                context=output["context"], question=output["question"]
            ),
        ),
        "relevance": call_llm(
            llm_client,
            question_relevance_critique_prompt.format(question=output["question"]),
        ),
        "standalone": call_llm(
            llm_client,
            question_standalone_critique_prompt.format(question=output["question"]),
        ),
    }
    try:
        for criterion, evaluation in evaluations.items():
            score, eval = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval,
                }
            )
    except Exception as e:
        continue

Generating critique for each QA couple...


100%|██████████| 10/10 [01:22<00:00,  8.21s/it]


Now let us filter out bad questions based on our critique agent scores:

In [40]:
import pandas as pd

pd.set_option("display.max_colwidth", None)

generated_questions = pd.DataFrame.from_dict(outputs)

print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    & (generated_questions["relevance_score"] >= 4)
    & (generated_questions["standalone_score"] >= 4)
]
print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

eval_dataset = datasets.Dataset.from_pandas(
    generated_questions, split="train", preserve_index=False
)

Evaluation dataset before filtering:


,question,answer,groundedness_score,relevance_score,standalone_score
0,How can you specify the maximum sequence length and device type when computing FrugalScore in Hugging Face?,"When computing FrugalScore using the `frugalscore.compute()` method, you can specify the maximum sequence length by setting the `max_length` parameter, and you can choose the device type by setting the `device` parameter to either ""gpu"" or ""cpu"". For example: `frugalscore.compute(predictions=['hello there'], references=['hello world'], max_length=64, device=""gpu"")`.",5,4,4
1,How can you compute accuracy with normalization turned off using the Hugging Face Evaluate library?,"In the Hugging Face Evaluate library, you can compute accuracy with normalization turned off by setting the `normalize` parameter to `False` when calling the `compute` method on an accuracy metric object. For example:\n\n```python\naccuracy_metric = evaluate.load(""accuracy"")\nresults = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0], normalize=False)\nprint(results)\n```\n\nThis will output the sum of correctly predicted samples instead of the proportion of correct predictions over total samples.",5,4,5
2,"How can NeRF-to-mesh conversion be applied, and what are its limitations in creating game-ready 3D assets?","NeRF-to-mesh conversion can be applied by using tools like NVlabs' instant-ngp, which facilitate transforming neural radiance fields (NeRFs) into mesh representations. However, the resulting meshes are not immediately game-ready and require significant manual post-processing and expertise, similar to photogrammetry techniques, to become usable in game development. Consequently, while NeRF-to-mesh can create high-fidelity assets, it's often not practical for quick and straightforward game asset creation.",4,1,5
3,How do you use datasets-cli to automatically generate feature types and sizes for a dataset in the Hugging Face Datasets library?,"To automatically generate feature types and sizes for a dataset with datasets-cli in the Hugging Face Datasets library, you can use the `datasets-cli test` command followed by the directory containing your dataset's loading script. This command will inspect your dataset and create a README.md with metadata, including feature types, dataset size, and number of examples for each split.",2,4,4
4,How can you modify the order of images within a Hugging Face dataset collection image gallery?,"To modify the order of images within a Hugging Face dataset collection image gallery, you can drag and drop the images to your desired position.",5,4,5
5,How can you configure Gradio UI components like Textbox and Checkbox to trigger specific actions on their state changes?,"In Gradio, you can set up UI components like Textbox and Checkbox to trigger actions on their state changes by using the `on_change` event. You define a function that specifies what should happen when the component's state changes. Then, you attach this function to the `on_change` event of the component, allowing for dynamic interactions on user input modifications.",2,4,5
6,"How can I use Gradio to create datasets with different types of components like images, markdown, and sliders?","In Gradio, you can create datasets with different types of components by using the `gr.Dataset` class, passing the appropriate component type to the `components` parameter. For example, use `gr.Dataset(components=[gr.Image()])` for images, `gr.Dataset(components=[gr.Markdown()])` for markdown, and `gr.Dataset(components=[gr.Slider()])` for sliders. Each dataset can include a list of sample values corresponding to the component's type to showcase different input possibilities or responses.",5,4,5
7,"How do you create multiple configurations for a dataset using the Hugging Face `datasets` library, and what role does `BuilderConfig` play in this process?","To create multiple configurations for a dataset using the Hugging Face `datasets` library, you need to 

Final evaluation dataset:


,question,answer,groundedness_score,relevance_score,standalone_score
0,How can you specify the maximum sequence length and device type when computing FrugalScore in Hugging Face?,"When computing FrugalScore using the `frugalscore.compute()` method, you can specify the maximum sequence length by setting the `max_length` parameter, and you can choose the device type by setting the `device` parameter to either ""gpu"" or ""cpu"". For example: `frugalscore.compute(predictions=['hello there'], references=['hello world'], max_length=64, device=""gpu"")`.",5,4,4
1,How can you compute accuracy with normalization turned off using the Hugging Face Evaluate library?,"In the Hugging Face Evaluate library, you can compute accuracy with normalization turned off by setting the `normalize` parameter to `False` when calling the `compute` method on an accuracy metric object. For example:\n\n```python\naccuracy_metric = evaluate.load(""accuracy"")\nresults = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0], normalize=False)\nprint(results)\n```\n\nThis will output the sum of correctly predicted samples instead of the proportion of correct predictions over total samples.",5,4,5
4,How can you modify the order of images within a Hugging Face dataset collection image gallery?,"To modify the order of images within a Hugging Face dataset collection image gallery, you can drag and drop the images to your desired position.",5,4,5
6,"How can I use Gradio to create datasets with different types of components like images, markdown, and sliders?","In Gradio, you can create datasets with different types of components by using the `gr.Dataset` class, passing the appropriate component type to the `components` parameter. For example, use `gr.Dataset(components=[gr.Image()])` for images, `gr.Dataset(components=[gr.Markdown()])` for markdown, and `gr.Dataset(components=[gr.Slider()])` for sliders. Each dataset can include a list of sample values corresponding to the component's type to showcase different input possibilities or responses.",5,4,5
7,"How do you create multiple configurations for a dataset using the Hugging Face `datasets` library, and what role does `BuilderConfig` play in this process?","To create multiple configurations for a dataset using the Hugging Face `datasets` library, you need to use the `BuilderConfig` class to define different configurations. Each configuration represents a different subset of the dataset. First, create a subclass of `BuilderConfig` for your dataset, including parameters like `data_url` and `metadata_urls` to specify where to download the data. Then, define your configurations in the `BUILDER_CONFIGS` list of your `GeneratorBasedBuilder` subclass. Each configuration should include a unique name, description, and the corresponding URLs for downloading its data and metadata. This allows you to handle different versions or subsets of a dataset, such as different data splits or categorically distinct datasets.",5,5,5
8,How can Intel's SigOpt be used to optimize performance settings for machine learning workloads on systems with multiple tuning parameters?,"Intel's SigOpt can be employed to optimize performance settings by using Bayesian optimization to efficiently explore the parameter space and find near-optimal configurations. This approach significantly reduces the number of experiments compared to a brute force search. By tuning parameters such as the number of cores, memory allocator, parallelism library, Transparent Huge Pages, and KMP block time parameter, SigOpt helps achieve performance close to the best possible while analyzing fewer configurations, thus saving time and computational resources.",5,4,5
9,How can you customize the cache directory location for the Hugging Face Hub cache-system?,You can customize the cache directory location for the Hugging Face Hub cache-system by using the `cache_dir` argument in methods or by setting the `HF_HOME` or `HF_HUB_CACHE` environment va

Now our synthetic evaluation dataset is complete! We can evaluate different RAG systems on this evaluation dataset.

We have generated only a few QA couples here to reduce time and cost. But let's kickstart the next part by loading a pre-generated dataset:

In [41]:
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train")

README.md:   0%|          | 0.00/893 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/289k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65 [00:00<?, ? examples/s]

# 2. Build our RAG System

### 2.1. Preprocessing documents to build our vector database

- In this part, __we split the documents from our knowledge base into smaller chunks__: these will be the snippets that are picked by the Retriever, to then be ingested by the Reader LLM as supporting elements for its answer.
- The goal is to build semantically relevant snippets: not too small to be sufficient for supporting an answer, and not too large too avoid diluting individual ideas.

Many options exist for text splitting:
- split every `n` words / characters, but this has the risk of cutting in half paragraphs or even sentences
- split after `n` words / character, but only on sentence boundaries
- **recursive split** tries to preserve even more of the document structure, by processing it tree-like way, splitting first on the largest units (chapters) then recursively splitting on smaller units (paragraphs, sentences).

To learn more about chunking, I recommend you read [this great notebook](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb) by Greg Kamradt.

[This space](https://huggingface.co/spaces/m-ric/chunk_visualizer) lets you visualize how different splitting options affect the chunks you get.

> In the following, we use Langchain's `RecursiveCharacterTextSplitter`.

💡 _To measure chunk length in our Text Splitter, our length function will not be the count of characters, but the count of tokens in the tokenized text: indeed, for subsequent embedder that processes token, measuring length in tokens is more relevant and empirically performs better._

In [42]:
from langchain_core.documents import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]

100%|██████████| 2647/2647 [00:00<00:00, 24562.06it/s]


In [43]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer


def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: str,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of size `chunk_size` characters and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

### 2.2. Retriever - embeddings 🗂️
The __retriever acts like an internal search engine__: given the user query, it returns the most relevant documents from your knowledge base.

> For the knowledge base, we use Langchain vector databases since __it offers a convenient [FAISS](https://github.com/facebookresearch/faiss) index and allows us to keep document metadata throughout the processing__.

🛠️ __Options included:__

- Tune the chunking method:
    - Size of the chunks
    - Method: split on different separators, use [semantic chunking](https://python.langchain.com/docs/modules/data_connection/document_transformers/semantic-chunker)...
- Change the embedding model

In [44]:
!pip install -qU langchain-community faiss-cpu

In [45]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
import os


def load_embeddings(
    langchain_docs: List[LangchainDocument],
    chunk_size: int,
    embedding_model_name: Optional[str] = "thenlper/gte-small",
) -> FAISS:
    """
    Creates a FAISS index from the given embedding model and documents. Loads the index directly if it already exists.

    Args:
        langchain_docs: list of documents
        chunk_size: size of the chunks to split the documents into
        embedding_model_name: name of the embedding model to use

    Returns:
        FAISS index
    """
    # load embedding_model
    embedding_model = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        multi_process=True,
        model_kwargs={"device": "cuda"},
        encode_kwargs={
            "normalize_embeddings": True
        },  # set True to compute cosine similarity
    )

    # Check if embeddings already exist on disk
    index_name = (
        f"index_chunk:{chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
    )
    index_folder_path = f"./data/indexes/{index_name}/"
    if os.path.isdir(index_folder_path):
        return FAISS.load_local(
            index_folder_path,
            embedding_model,
            distance_strategy=DistanceStrategy.COSINE,
            allow_dangerous_deserialization=True  # <--- ADD THIS LINE
        )

    else:
        print("Index not found, generating it...")
        docs_processed = split_documents(
            chunk_size,
            langchain_docs,
            embedding_model_name,
        )
        knowledge_index = FAISS.from_documents(
            docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
        )
        knowledge_index.save_local(index_folder_path)
        return knowledge_index

### 2.3. Reader - LLM 💬

In this part, the __LLM Reader reads the retrieved documents to formulate its answer.__

🛠️ Here we tried the following options to improve results:
- Switch reranking on/off
- Change the reader model

In [46]:
RAG_PROMPT_TEMPLATE = """
<|system|>
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
</s>
<|assistant|>
"""

In [47]:
!pip install langchain_huggingface

In [48]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 3.1 MB/s eta 0:00:00


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# It is best practice to set your key as an environment variable
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# 1. Initialize the ChatOpenAI model
READER_LLM = ChatOpenAI(
    model="gpt-4o",
    openai_api_key="your-api-key",
    max_tokens=512,
    temperature=0.1,
    model_kwargs={"top_p": 0.9}
)

# 2. Maintain your specialized patch for callable behavior
def chat_patch(self, prompt, **kwargs):
    response = self.invoke([HumanMessage(content=prompt)])
    return response.content

# Apply the patch to the class
ChatOpenAI.__call__ = chat_patch


/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'top_p'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


In [53]:
from ragatouille import RAGPretrainedModel
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM


def answer_with_rag(
    question: str,
    llm: LLM,
    knowledge_index: VectorStore,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 7,
) -> Tuple[str, List[LangchainDocument]]:
    """Answer a question using RAG with the given knowledge index."""
    # Gather documents with retriever
    relevant_docs = knowledge_index.similarity_search(
        query=question, k=num_retrieved_docs
    )
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

    # Optionally rerank results
    if reranker:
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)]
    )

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    answer = llm.invoke(final_prompt)

    return answer, relevant_docs

/tmp/ipython-input-3993874634.py:1: UserWarning: 
********************************************************************************
RAGatouille WARNING: Future Release Notice
--------------------------------------------
RAGatouille version 0.0.10 will be migrating to a PyLate backend 
instead of the current Stanford ColBERT backend.
PyLate is a fully mature, feature-equivalent backend, that greatly facilitates compatibility.
However, please pin version <0.0.10 if you require the Stanford ColBERT backend.
********************************************************************************
  from ragatouille import RAGPretrainedModel


# 3. Benchmarking the RAG system

The RAG system and the evaluation datasets are now ready. The last step is to judge the RAG system's output on this evaluation dataset.

To this end, __we setup a judge agent__. ⚖️🤖

Out of [the different RAG evaluation metrics](https://docs.ragas.io/en/latest/concepts/metrics/index.html), we choose to focus only on Answer Correctness since it is the best end-to-end metric of our system's performance.

> We use GPT4 as a judge for its empirically good performance, but you could try with other models such as [kaist-ai/prometheus-13b-v1.0](https://huggingface.co/kaist-ai/prometheus-13b-v1.0) or [BAAI/JudgeLM-33B-v1.0](https://huggingface.co/BAAI/JudgeLM-33B-v1.0).

💡 _In the evaluation prompt, we give a detailed description each metric on the scale 1-5, as is done in [Prometheus's prompt template](https://huggingface.co/kaist-ai/prometheus-13b-v1.0): this helps the model ground its metric precisely. If instead you give the judge LLM a vague scale to work with, the outputs will not be consistent enough between different examples._

💡 _Again, prompting the LLM to output rationale before giving its final score gives it more tokens to help it formalize and elaborate a judgement._

In [54]:
from langchain_core.language_models import BaseChatModel

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    llm,
    knowledge_index: VectorStore,
    output_file: str,
    reranker: Optional[RAGPretrainedModel] = None,
    verbose: Optional[bool] = True,
    test_settings: Optional[str] = None,  # To document the test settings used
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    try:  # load previous generations if they exist
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []

    for example in tqdm(eval_dataset):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        answer, relevant_docs = answer_with_rag(
            question, llm, knowledge_index, reranker=reranker
        )
        # 1. Extract string from AIMessage if necessary
        generated_answer = answer.content if hasattr(answer, 'content') else str(answer)

        # 2. Extract page_content from Document objects
        retrieved_docs_content = [
            doc.page_content if hasattr(doc, 'page_content') else str(doc)
            for doc in relevant_docs
        ]

        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            # "generated_answer": answer,
            # "retrieved_docs": [doc for doc in relevant_docs],
            "generated_answer": generated_answer,
            "retrieved_docs": retrieved_docs_content,
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        with open(output_file, "w") as f:
            json.dump(outputs, f)

In [55]:
EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain_core.messages import HumanMessage, SystemMessage


evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

In [ ]:
from langchain_community.chat_models import ChatOpenAI

OPENAI_API_KEY = "your-api-key"

eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0, openai_api_key=OPENAI_API_KEY)
evaluator_name = "GPT4"


def evaluate_answers(
    answer_path: str,
    eval_chat_model,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    """Evaluates generated answers. Modifies the given answer file in place for better checkpointing."""
    answers = []
    if os.path.isfile(answer_path):  # load previous generations if they exist
        answers = json.load(open(answer_path, "r"))

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_result = eval_chat_model.invoke(eval_prompt)
        feedback, score = [
            item.strip() for item in eval_result.content.split("[RESULT]")
        ]
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with open(answer_path, "w") as f:
            json.dump(answers, f)

/tmp/ipython-input-1844943501.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import ChatOpenAI``.
  eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0, openai_api_key=OPENAI_API_KEY)


🚀 Let's run the tests and evaluate answers!👇

In [58]:
# READER_MODEL_NAME = "mistralai"
READER_MODEL_NAME = "gpt4"
if not os.path.exists("./output"):
    os.mkdir("./output")

for chunk_size in [200]:  # Add other chunk sizes (in tokens) as needed
    for embeddings in ["thenlper/gte-small"]:  # Add other embeddings as needed
        for rerank in [True, False]:
            settings_name = f"chunk:{chunk_size}_embeddings:{embeddings.replace('/', '~')}_rerank:{rerank}_reader-model:{READER_MODEL_NAME}"
            output_file_name = f"./output/rag_{settings_name}.json"

            print(f"Running evaluation for {settings_name}:")

            print("Loading knowledge base embeddings...")
            knowledge_index = load_embeddings(
                RAW_KNOWLEDGE_BASE,
                chunk_size=chunk_size,
                embedding_model_name=embeddings,
            )

            print("Running RAG...")
            reranker = (
                RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
                if rerank
                else None
            )
            run_rag_tests(
                eval_dataset=eval_dataset,
                llm=READER_LLM,
                knowledge_index=knowledge_index,
                output_file=output_file_name,
                reranker=reranker,
                verbose=False,
                test_settings=settings_name,
            )

            print("Running evaluation...")
            evaluate_answers(
                output_file_name,
                eval_chat_model,
                evaluator_name,
                evaluation_prompt_template,
            )

Running evaluation for chunk:200_embeddings:thenlper~gte-small_rerank:True_reader-model:mistralai:
Loading knowledge base embeddings...
Running RAG...


100%|██████████| 65/65 [00:00<00:00, 7314.21it/s]


Running evaluation...


100%|██████████| 65/65 [00:00<00:00, 646042.09it/s]


Running evaluation for chunk:200_embeddings:thenlper~gte-small_rerank:False_reader-model:mistralai:
Loading knowledge base embeddings...
Running RAG...


100%|██████████| 65/65 [00:00<00:00, 6502.95it/s]


Running evaluation...


100%|██████████| 65/65 [01:21<00:00,  1.26s/it]


### Inspect results

In [77]:
import glob

outputs = []
for file in glob.glob("./output/*.json"):
    output = pd.DataFrame(json.load(open(file, "r")))
    output["settings"] = file
    outputs.append(output)
result = pd.concat(outputs)

In [78]:
result["eval_score_GPT4"] = result["eval_score_GPT4"].apply(
    lambda x: int(x) if isinstance(x, str) else 1
)
result["eval_score_GPT4"] = (result["eval_score_GPT4"] - 1) / 4

In [79]:
average_scores = result.groupby("settings")["eval_score_GPT4"].mean()
average_scores.sort_values()

,eval_score_GPT4
settings,
./output/rag_chunk:200_embeddings:thenlper~gte-small_rerank:False_reader-model:gpt4.json,0.853846
./output/rag_chunk:200_embeddings:thenlper~gte-small_rerank:True_reader-model:gpt4.json,0.884615


In [80]:
import plotly.express as px

# 1. Chuẩn bị dữ liệu
df_plot = average_scores.reset_index()
df_plot.columns = ['settings', 'eval_score']
df_plot['eval_score_pct'] = df_plot['eval_score'] * 100

# 2. HÀM RÚT GỌN TÊN (Shorten Names)
def shorten_settings(name):
    # Loại bỏ tiền tố và hậu tố file
    name = name.replace("./output/", "").replace(".json", "")
    # Thay thế các cụm từ dài bằng ký hiệu ngắn hơn
    name = name.replace("rag_chunk:200_embeddings:thenlper~gte-small", "")
    name = name.replace("_reader-model:", " | Model:")
    name = name.replace("_rerank:", "Rerank:")
    return name

df_plot['short_settings'] = df_plot['settings'].apply(shorten_settings)

# 3. Tạo biểu đồ với tên đã rút gọn
fig = px.bar(
    df_plot,
    x="short_settings",      # Sử dụng cột tên đã rút gọn
    y="eval_score_pct",
    color="eval_score_pct",
    # Thêm hover_data để khi di chuột vào vẫn xem được đường dẫn gốc đầy đủ
    hover_data={"settings": True, "short_settings": False},
    labels={
        "eval_score_pct": "Accuracy (%)",
        "short_settings": "Config Summary",
    },
    color_continuous_scale="bluered",
    text="eval_score_pct"
)

# 4. Tinh chỉnh Layout
fig.update_layout(
    width=1000,
    height=600,
    title="<b>RAG Performance by Configuration</b>",
    xaxis_title="Settings",
    yaxis_title="Accuracy (%)",
    yaxis_range=[0, 110],
    font=dict(size=14),
    margin=dict(b=150), # Giảm lề dưới vì tên đã ngắn hơn
)


fig.update_traces(
    texttemplate='%{text:.1f}%',
    textposition='outside'
)

fig.update_coloraxes(showscale=False)
fig.show()

## Example results

Let us load the results that I obtained by tweaking the different options available in this notebook.
For more detail on why these options could work or not, see the notebook on [advanced_RAG](advanced_rag).

As you can see in the graph below, some tweaks do not bring any improvement, some give huge performance boosts.

➡️ ___There is no single good recipe: you should try several different directions when tuning your RAG systems.___


In [62]:
import plotly.express as px

scores = datasets.load_dataset("m-ric/rag_scores_cookbook", split="train")
scores = pd.Series(scores["score"], index=scores["settings"])

README.md:   0%|          | 0.00/319 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5 [00:00<?, ? examples/s]

In [63]:
fig = px.bar(
    scores,
    color=scores,
    labels={
        "value": "Accuracy",
        "settings": "Configuration",
    },
    color_continuous_scale="bluered",
)
fig.update_layout(
    width=1000,
    height=600,
    barmode="group",
    yaxis_range=[0, 100],
    title="<b>Accuracy of different RAG configurations</b>",
    xaxis_title="RAG settings",
    font=dict(size=15),
)
fig.layout.yaxis.ticksuffix = "%"
fig.update_coloraxes(showscale=False)
fig.update_traces(texttemplate="%{y:.1f}", textposition="outside")
fig.show()

<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_settings_accuracy.png" height="500" width="800">

As you can see, these had varying impact on performance. In particular, tuning the chunk size is both easy and very impactful.

But this is our case: your results could be very different: now that you have a robust evaluation pipeline, you can set on to explore other options! 🗺️